#### <b>Managing the conversation History </b>

one important concept to understand when building chatbots is how to manage conversation history.if left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the llm. therefore, it is important to add a step that limits the size of the messages you are passing in.

- <b>trim_messages</b> : helper to reduce how many messages we're sending to the model. the trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [30]:
import os
from dotenv import load_dotenv 
from rich import print 
load_dotenv(dotenv_path="../.env")

from langchain_groq import ChatGroq 
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm_openai=ChatOpenAI(model="gpt-5-mini-2025-08-07")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="gemma2-9b-it")

In [33]:
from langchain_core.messages import SystemMessage,trim_messages,HumanMessage,AIMessage
trimmer=trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [34]:
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! i am abhijit"),
    AIMessage(content="hi"),
    HumanMessage(content="i like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
     HumanMessage(content=" having fun ?"),
    AIMessage(content="yes!"),
]

In [35]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi! i am abhijit', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' having fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [36]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",("You are helpful assistant.Answer all the questions to the best of your ability in {language}")),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [37]:
from operator import itemgetter 

from langchain_core.runnables import RunnablePassthrough 

chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
        |prompt
        |llm_openai
)

In [ ]:
print(chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What ice cream do i like")],
    "language":"english"
    }
))

AIMessage(
    content='You like vanilla ice cream.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 79,
            'prompt_tokens': 121,
            'total_tokens': 200,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 64,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-5-mini-2025-08-07',
        'system_fingerprint': None,
        'id': 'chatcmpl-C73bHcECt6XbMRb9I7szmQTb3SIiV',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--d2c92bb7-8260-409c-82b9-30f37ec51300-0',
    usage_metadata={
        'input_tokens': 121,
        'output_tokens': 79,
        'total_tokens': 200,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 64}
    }
)

In [41]:
# let's warp this in the message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response=chain.invoke(
    {
    "messages":messages+[HumanMessage(content="what is my name ? ")],
    "language":"english"
    },
    config=config
)

In [43]:
print(response)

AIMessage(
    content='You told me your name is Abhijit.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 148,
            'prompt_tokens': 121,
            'total_tokens': 269,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 128,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-5-mini-2025-08-07',
        'system_fingerprint': None,
        'id': 'chatcmpl-C73gyVB8lJdlGVOfdn1fyoqG8PS8F',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--e78b340f-ff1e-430e-b72e-4526fb414045-0',
    usage_metadata={
        'input_tokens': 121,
        'output_tokens': 148,
        'total_tokens': 269,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 128}
    }
)